In [2]:
import pandas as pd

df_ann = pd.read_csv('./annotated_pagos/areiospagos_annotated.csv')

In [3]:
df_ann

,Decision,Label
0,Decision_0,1
1,Decision_1,0
2,Decision_2,1
3,Decision_3,1
4,Decision_4,1
...,...,...
2995,Decision_2995,1
2996,Decision_2996,1
2997,Decision_2997,1
2998,Decision_2998,1


In [4]:
df_ann['Label'].value_counts()

Label
1    2450
0     550
Name: count, dtype: int64

Creating a new balanced dataset to work with for Areios Pagos


In [5]:
df_0 = df_ann[df_ann['Label'] == 0]
df_1 = df_ann[df_ann['Label'] == 1]

df_0_sample = df_0.sample(500)
df_1_sample = df_1.sample(550)

df_resampled_ap = pd.concat([df_0_sample, df_1_sample])

# Shuffle the dataset
df_resampled_ap = df_resampled_ap.sample(frac=1).reset_index(drop=True)

# Check the distribution of labels
df_resampled_ap['Label'].value_counts()


Label
1    550
0    500
Name: count, dtype: int64

In [6]:
df_resampled_ap

,Decision,Label
0,Decision_1088,0
1,Decision_2020,0
2,Decision_2402,0
3,Decision_157,0
4,Decision_168,1
...,...,...
1045,Decision_302,1
1046,Decision_1876,1
1047,Decision_1796,0
1048,Decision_2629,0


In [7]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import logging  # Setting up the loggings to monitor gensim
import re
import unicodedata as ud
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata

def preprocess_text(file_path):
    with open(file_path, 'r') as file:
        # Read the text from the file
        text = file.read()
        # Convert to lowercase
        text = text.lower()
        # Remove accents
        text = ''.join(char for char in unicodedata.normalize('NFKD', text) if not unicodedata.combining(char))
        # Remove punctuations
        text = re.sub(r'[^\w\s]', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove English characters
        text = re.sub(r'[a-zA-Z]', '', text)
        # Remove all special chars
        text = text.replace('_', '')
        # Transforn to list of words
        text = text.replace('\n', ' ').split(' ')
    return text
stop_words = preprocess_text('stopwords.txt')



In [9]:
print(df_ann.columns)

Index(['Decision', 'Label'], dtype='object')


In [10]:
df = pd.read_csv('areiospagos_annotation/areiospagos_annottated.csv')


In [11]:
print(df.columns)

Index(['Decision', 'Label'], dtype='object')


In [12]:
df


,Decision,Label
0,ΑΡΙΘΜΟΣ 970/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
1,Αριθμός 1692/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,0
2,Αριθμός 92/2013 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ Π...,1
3,Αριθμός 1502/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
4,Αριθμός 1501/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
...,...,...
2995,ΑΡΙΘΜΟΣ 350/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2996,Αριθμός 354/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2997,Αριθμός 332/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2998,ΑΡΙΘΜΟΣ 328/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1


In [13]:
df['processed_text'] = df['Decision'].apply(preprocess_text)


OSError: [Errno 63] File name too long: 'ΑΡΙΘΜΟΣ 970/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ΠΑΓΟΥ\n Z\' ΠΟΙΝΙΚΟ ΤΜΗΜΑ  \nΣυγκροτήθηκε από τους Δικαστές: Γρηγόριο Μάμαλη, Αντιπρόεδρο Αρείου Πάγου, Νικόλαο Ζαΐρη - Εισηγητή, Νικόλαο Κωνσταντόπουλο, Παναγιώτη Ρουμπή και Κωνσταντίνο Φράγκο, Αρεοπαγίτες.\n \nΣυνήλθε σε δημόσια συνεδρίαση στο Κατάστημά του στις 12 Νοεμβρίου 2008, με την παρουσία του Αντεισαγγελέα του Αρείου Πάγου Ανδρέα Ζύγουρα (γιατί κωλύεται ο Εισαγγελέας) και του Γραμματέα Χρήστου Πήτα, για να δικάσει την αίτηση των αναιρεσειόντων - κατηγορουμένων: 1. Χ1 που παραστάθηκε με τον πληρεξούσιο δικηγόρο του Αλέξιο Κούγια και 2. Χ2 και ήδη κρατούμενου στην Κλειστή Φυλακή ..., που εκπροσωπήθηκε από την πληρεξούσια δικηγόρο του Αλεξάνδρα Μαύρου - Τσάκου, περί αναιρέσεως της 1/2007 αποφάσεως του Πενταμελούς Εφετείου Λαρίσης. \nΜε πολιτικώς ενάγουσα την Ανώνυμη Τραπεζική Εταιρεία με την επωνυμία "ΕΘΝΙΚΗ ΤΡΑΠΕΖΑ ΤΗΣ ΕΛΛΑΔΟΣ Α.Ε.", που εδρεύει στην Αθήνα και εκπροσωπείται νόμιμα και που στο

## TF-IDF VECTORIZER ##
TF-IDF Vectorization:

It uses TfidfVectorizer from scikit-learn to convert the text data (df_resampled['script']) into a matrix of TF-IDF features.
The parameters used in TfidfVectorizer are:

norm='l2': This normalizes the vectors to have an L2 norm (unit norm).

stop_words=stop_words: This removes common stop words (as defined by the stop_words variable) from the text before vectorizing.

max_features=5000: Limits the vocabulary to the 5000 most frequent terms.

Transformation:

The fit_transform method is applied to the text data (df_resampled['script']), which learns the vocabulary and IDF (Inverse Document Frequency) values from the data, and then transforms the text data into a TF-IDF weighted term-document matrix.
This matrix is then converted into a dense array using .toarray().

Creating a DataFrame:

The TF-IDF matrix is then converted into a Pandas DataFrame (data_resampled), with the same index as the original DataFrame (df_resampled.index) and with columns named after the terms in the vocabulary (tfidf.get_feature_names_out()).
Appending the Label:

The 'Label' column from the original df_resampled DataFrame is added to the data_resampled DataFrame, aligning by index.





In [14]:
# Load your custom stop words from 'stopwords.txt'
def preprocess_text(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = [line.strip() for line in file]
    return stopwords

stop_words = preprocess_text('stopwords.txt')

# Initialize TfidfVectorizer with your stop words and set max_features
tfidf = TfidfVectorizer(norm='l2', stop_words=stop_words, max_features=5000)

# Apply the TF-IDF transformation on the 'script' column (or rename if necessary)
tfidf_matrix = tfidf.fit_transform(df_ann['Decision']).toarray()

# Create a new DataFrame with TF-IDF features and add the 'Label' column
data_ann = pd.DataFrame(tfidf_matrix, index=df_ann.index, columns=tfidf.get_feature_names_out())
data_ann['Label'] = df_ann['Label'].values


/Users/thodorismexis/anaconda3/envs/decisions/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['αδιακοπα', 'αι', 'ακομα', 'ακομη', 'ακριβως', 'αληθεια', 'αληθινα', 'αλλα', 'αλλαχου', 'αλλες', 'αλλη', 'αλλην', 'αλλης', 'αλλιως', 'αλλιωτικα', 'αλλο', 'αλλοι', 'αλλοιως', 'αλλοιωτικα', 'αλλον', 'αλλος', 'αλλοτε', 'αλλου', 'αλλους', 'αλλων', 'αμα', 'αμεσα', 'αμεσως', 'αν', 'ανα', 'αναμεσα', 'αναμεταξυ', 'ανευ', 'αντι', 'αντιπερα', 'αντις', 'ανω', 'ανωτερω', 'αξαφνα', 'απ', 'απεναντι', 'απο', 'αποψε', 'αρα', 'αραγε', 'αργα', 'αργοτερο', 'αριστερα', 'αρκετα', 'αρχικα', 'ας', 'αυριο', 'αυτα', 'αυτες', 'αυτη', 'αυτην', 'αυτης', 'αυτο', 'αυτοι', 'αυτον', 'αυτος', 'αυτου', 'αυτους', 'αυτων', 'αφοτου', 'αφου', 'βεβαια', 'βεβαιοτατα', 'γι', 'για', 'γρηγορα', 'γυρω', 'δα', 'δε', 'δεινα', 'δεν', 'δεξια', 'δηθεν', 'δηλαδη', 'δι', 'δια', 'διαρκως', 'δικα', 'δικο', 'δ

In [19]:
df


,Decision,Label
0,ΑΡΙΘΜΟΣ 970/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
1,Αριθμός 1692/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,0
2,Αριθμός 92/2013 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ Π...,1
3,Αριθμός 1502/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
4,Αριθμός 1501/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
...,...,...
2995,ΑΡΙΘΜΟΣ 350/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2996,Αριθμός 354/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2997,Αριθμός 332/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2998,ΑΡΙΘΜΟΣ 328/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1


In [20]:
df = df.reset_index(drop=True)
df_ann = df_ann.reset_index(drop=True)

# Add the text and label columns to a new DataFrame
df_combined = pd.DataFrame({
    'text': df['Decision'],  # Replace with the actual text column name in `df`
    'label': df_ann['Label']      # Replace 'Label' with the label column name in `df_ann`
})

In [21]:
df_combined

,text,label
0,ΑΡΙΘΜΟΣ 970/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
1,Αριθμός 1692/2009 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,0
2,Αριθμός 92/2013 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ Π...,1
3,Αριθμός 1502/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
4,Αριθμός 1501/2018 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ...,1
...,...,...
2995,ΑΡΙΘΜΟΣ 350/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2996,Αριθμός 354/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2997,Αριθμός 332/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1
2998,ΑΡΙΘΜΟΣ 328/2011 \n\nΤΟ ΔΙΚΑΣΤΗΡΙΟ ΤΟΥ ΑΡΕΙΟΥ ...,1


In [22]:
df_combined['label'].value_counts()


label
1    2450
0     550
Name: count, dtype: int64

In [23]:
# Split the dataset by label
df_0 = df_combined[df_combined['label'] == 0]
df_1 = df_combined[df_combined['label'] == 1]

# Sample equal amounts from each class
df_0_sample = df_0.sample(500, random_state=42)
df_1_sample = df_1.sample(500, random_state=42)

# Combine the samples
df_resampled = pd.concat([df_0_sample, df_1_sample])

# Shuffle the dataset
df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the distribution of labels
print(df_resampled['label'].value_counts())

label
1    500
0    500
Name: count, dtype: int64


In [31]:
df_resampled

,00,000,01,02,03,04,05,06,07,08,...,όψη,ύλην,ύπαρξη,ύπαρξης,ύστερα,ύψος,ύψους,ώρα,ώρες,ώστε
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.010802,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.047153,0.003861,0.0,0.0,0.000000,0.0,0.0,0.007642,0.0,0.0,...,0.000000,0.0,0.003713,0.0,0.005121,0.016308,0.0,0.018661,0.183419,0.014056
3,0.000000,0.135882,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.008245
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.073531,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.030048,0.0,0.000000,0.0,0.010838,0.000000,0.0,0.000000,0.000000,0.007436
996,0.011083,0.083484,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.035090,0.000000,0.000000
997,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
998,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
